In [1]:
! pip3 install -qU langchain-upstage python-dotenv langchain_core markdown langchain_community langchain_text_splitters langchain_chroma


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [7]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [3]:
import markdown

# Convert markdown file to HTML
def convert_markdown_to_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    html = markdown.markdown(content)
    return html

file_path = 'apply-home.md'
markdown_string = ''
with open(file_path, 'r', encoding='utf-8') as file:
    markdown_string = file.read()
print(len(markdown_string))

109097


In [4]:
from langchain_community.retrievers import BM25Retriever
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=100, language=Language.MARKDOWN
)
splits = text_splitter.split_text(markdown_string)

In [12]:
import re

def split_markdown_by_h2(content):
    # Use regex to split the content by `##` titles
    sections = re.split(r'(## .+)', content)
    # Initialize an empty list to hold the final sections
    split_content = []
    
    # Combine the titles with their corresponding content
    for i in range(1, len(sections), 2):
        section_title = sections[i]
        section_content = sections[i + 1] if (i + 1) < len(sections) else ''
        split_content.append(section_title + section_content)
    
    return split_content

section = split_markdown_by_h2(markdown_string)
print(section[0][:100])

## 청약주택
#


In [9]:
from langchain.docstore.document import Document
from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings

section_docs = [Document(page_content=text) for text in section]

# 3. Embed & indexing
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=UpstageEmbeddings(model="solar-embedding-1-large"),
)

AttributeError: 'str' object has no attribute 'page_content'

In [ ]:
retriever = vectorstore.as_retriever()
result_docs = retriever.invoke("청약 자격")
print(len(result_docs))
print(result_docs[0].page_content[:100])

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context. 
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [10]:
query = '우리가 청약할 수 있는 분양공고인지 알고 싶어'
chain.invoke({"question": query, "Context": context_docs})

'문맥에서 "사전청약에 당첨된 이후 상속 혼인의 사유로 주택을 추가로 소유하게 되는 경우 사전당첨 자격 인정이 가능한지?"에 대한 답변은 없습니다.'